In [366]:
import pandas as pd

# df = pd.read_csv('story_data.csv')

# opening our current file of results
df = pd.read_csv('story_data_until886.csv')

In [367]:
df.shape

(887, 12)

For now, we have 887 stories and 12 features

In [368]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   url             887 non-null    object
 1   story_id        887 non-null    object
 2   soup            887 non-null    object
 3   title           887 non-null    object
 4   author          887 non-null    object
 5   contest_num     887 non-null    object
 6   won_contest     11 non-null     object
 7   categories      887 non-null    object
 8   num_likes       887 non-null    int64 
 9   num_comments    887 non-null    int64 
 10  story_html      887 non-null    object
 11  date_published  887 non-null    object
dtypes: int64(2), object(10)
memory usage: 83.3+ KB


'won_contest' only has a value if the story was a winner or shortlisted

In [369]:
df.head()

,url,story_id,soup,title,author,contest_num,won_contest,categories,num_likes,num_comments,story_html,date_published
0,https://blog.reedsy.com/short-story/n1pl7n/,n1pl7n,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,11:59,Krystal Pepper,"['Contest', '#137']",NaN,"['Crime', 'Fiction', 'Thriller']",17,3,"<article class=""font-alt submission-content sp...","March 11, 2022 17:27"
1,https://blog.reedsy.com/short-story/yv1ahb/,yv1ahb,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,Want Ads,Nina Wishnat,"['Contest', '#30']",NaN,"['Contemporary', 'Fiction']",1,0,"<article class=""font-alt submission-content sp...","February 28, 2020 15:27"
2,https://blog.reedsy.com/short-story/kv1cr7/,kv1cr7,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,To be wrong because of sincerity...,Lis Lovén,"['Contest', '#102']",NaN,"['Black', 'Contemporary', 'Fiction']",12,0,"<article class=""font-alt submission-content sp...","July 10, 2021 14:21"
3,https://blog.reedsy.com/short-story/v2nqtq/,v2nqtq,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,"The Shiva, 1955",Tammy Kl,"['Contest', '#100']",NaN,"['American', 'Coming', 'of', 'Age', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 01, 2021 18:13"
4,https://blog.reedsy.com/short-story/nptt18/,nptt18,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,The Monster of Greentree,Best Christopher,"['Contest', '#102']",NaN,"['Coming', 'of', 'Age', 'Adventure', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 17, 2021 01:23"


In [370]:
df.describe()

,num_likes,num_comments
count,887.000000,887.000000
mean,14.512965,4.717024
std,21.576877,21.759873
min,0.000000,0.000000
25%,8.000000,0.000000
50%,10.000000,1.000000
75%,14.000000,3.000000
max,460.000000,420.000000


two numeric features: 'num_likes' is target feature for regression problem

TODO - turn date into datetime object

features to extract:
tokenize words
word related features


extracting some word-related features:

In [371]:
from bs4 import BeautifulSoup
from nltk.tokenize import wordpunct_tokenize, sent_tokenize, word_tokenize

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Batra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [372]:
def separate_words_and_punct(tokenized_text):
    words = []
    punctuation = []
    for i, value in enumerate(tokenized_text):
        if len(value) == 1 and not value.isalpha():
            punctuation.append(i)
        else:
            words.append(i)
#     print('words index:\n', words)
#     print('\nwords:\n', tokenized_text[words])
    return tokenized_text[words], tokenized_text[punctuation]

In [373]:
def tokenize_story(row):
    html = row['story_html']
    soup = BeautifulSoup(html, "html.parser")
    story = soup.get_text()
    story_sent = sent_tokenize(story)
    num_sent = len(story_sent)
    story_word = word_tokenize(story)
    words, punctuation = separate_words_and_punct(np.array(story_word))
    num_words = len(words)
    unique_words = set(words)
    num_unique_words = len(unique_words)
    unique_words_percent = num_unique_words / num_words
    feat_dict = {'num_sent': num_sent, 'words': words, 'num_words': num_words, 
                         'unique_words': unique_words, 'num_unique_words': num_unique_words, 
                         'unique_words_percent': unique_words_percent, 'punctuation': punctuation}
    return pd.DataFrame.from_dict(feat_dict, orient='index').transpose()

In [374]:
new_feats = ('num_sent', 'words', 'num_words', 'unique_words', 'num_unique_words', 'unique_words_percent','punctuation')
results = df.apply(tokenize_story, axis=1)

In [376]:
feats_df = pd.DataFrame()
for sub in results:
    feats_df = pd.concat([feats_df, sub])
feats_df = feats_df.reset_index()
feats_df.drop('index', axis=1, inplace=True)
feats_df.head()

,num_sent,words,num_words,unique_words,num_unique_words,unique_words_percent,punctuation
0,125,"[11:59, I, shoot, straight, out, of, bed, to, ...",2645,"{lot, on, I, detective, leave, actually, word,...",774,0.292628,"[., ., ., ., ., ,, ,, ., ,, ,, ., ,, ., ,, ., ..."
1,87,"[Mia, s, room, was, pretty, much, bare, She, l...",1453,"{firmly, better, on, I, made, elevator, With, ...",619,0.426015,"[’, ., ,, ., ., ,, ., ., ., ., ., “, ?, ’, ,, ..."
2,150,"[Lin, had, to, figure, out, things, because, o...",2008,"{loads, better, on, creature, word, made, cham...",676,0.336653,"[,, ,, ., ,, ,, ., ., ,, ., ,, ,, ., ,, ,, ., ..."
3,153,"[The, Shiva, 1955, For, the, rest, of, her, li...",2735,"{peeked, on, I, word, fell, polite, made, hold...",994,0.363437,"[,, ,, ., ,, ., –, ,, ,, ,, ’, ., (, !, ), ,, ..."
4,186,"[The, wind, whipped, through, my, hair, as, I,...",2773,"{lot, better, insane, on, I, him—which, word, ...",863,0.311215,"[’, ,, ., ., ,, ,, ., ,, ,, ., ,, ., ,, ,, ’, ..."


In [377]:
feats_df.shape, df.shape

((887, 7), (887, 12))

7 new features extracted

In [378]:
new_df = pd.concat([df, feats_df], axis=1)
new_df

,url,story_id,soup,title,author,contest_num,won_contest,categories,num_likes,num_comments,story_html,date_published,num_sent,words,num_words,unique_words,num_unique_words,unique_words_percent,punctuation
0,https://blog.reedsy.com/short-story/n1pl7n/,n1pl7n,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,11:59,Krystal Pepper,"['Contest', '#137']",NaN,"['Crime', 'Fiction', 'Thriller']",17,3,"<article class=""font-alt submission-content sp...","March 11, 2022 17:27",125,"[11:59, I, shoot, straight, out, of, bed, to, ...",2645,"{lot, on, I, detective, leave, actually, word,...",774,0.292628,"[., ., ., ., ., ,, ,, ., ,, ,, ., ,, ., ,, ., ..."
1,https://blog.reedsy.com/short-story/yv1ahb/,yv1ahb,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,Want Ads,Nina Wishnat,"['Contest', '#30']",NaN,"['Contemporary', 'Fiction']",1,0,"<article class=""font-alt submission-content sp...","February 28, 2020 15:27",87,"[Mia, s, room, was, pretty, much, bare, She, l...",1453,"{firmly, better, on, I, made, elevator, With, ...",619,0.426015,"[’, ., ,, ., ., ,, ., ., ., ., ., “, ?, ’, ,, ..."
2,https://blog.reedsy.com/short-story/kv1cr7/,kv1cr7,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,To be wrong because of sincerity...,Lis Lovén,"['Contest', '#102']",NaN,"['Black', 'Contemporary', 'Fiction']",12,0,"<article class=""font-alt submission-content sp...","July 10, 2021 14:21",150,"[Lin, had, to, figure, out, things, because, o...",2008,"{loads, better, on, creature, word, made, cham...",676,0.336653,"[,, ,, ., ,, ,, ., ., ,, ., ,, ,, ., ,, ,, ., ..."
3,https://blog.reedsy.com/short-story/v2nqtq/,v2nqtq,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,"The Shiva, 1955",Tammy Kl,"['Contest', '#100']",NaN,"['American', 'Coming', 'of', 'Age', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 01, 2021 18:13",153,"[The, Shiva, 1955, For, the, rest, of, her, li...",2735,"{peeked, on, I, word, fell, polite, made, hold...",994,0.363437,"[,, ,, ., ,, ., –, ,, ,, ,, ’, ., (, !, ), ,, ..."
4,https://blog.reedsy.com/short-story/nptt18/,nptt18,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,The Monster of Greentree,Best Christopher,"['Contest', '#102']",NaN,"['Coming', 'of', 'Age', 'Adventure', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 17, 2021 01:23",186,"[The, wind, whipped, through, my, hair, as, I,...",2773,"{lot, better, insane, on, I, him—which, word, ...",863,0.311215,"[’, ,, ., ., ,, ,, ., ,, ,, ., ,, ., ,, ,, ’, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,https://blog.reedsy.com/short-story/5h2534/,5h2534,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,From Fraud to Friend,Kita Munesane,"['Contest', '#127']",NaN,['Fiction'],10,0,"<article class=""font-alt submission-content sp...","January 07, 2022 01:55",49,"[As, the, gullible, crowd, screamed, as, Paul,...",1138,"{on, I, Prime, fell, made, hold, gain, hero, W...",457,0.401582,"[,, ,, ., ,, ., ,, ., ., ,, ,, ,, ,, ., ,, ,, ..."
883,https://blog.reedsy.com/short-story/xq3yj9/,xq3yj9,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,Homemade Marmelade,Anoella Lowin,"['Contest', '#140']",NaN,"['Fiction', 'Drama']",10,0,"<article class=""font-alt submission-content sp...","April 07, 2022 23:25",229,"[When, somebody, knocks, on, my, bedroom, door...",2606,"{lot, better, on, I, leave, word, whose, made,...",687,0.263622,"[,, ., ., ,, ,, ., ., ., ., ?, ., ,, ., ., ,, ..."
884,https://blog.reedsy.com/short-story/62rs3a/,62rs3a,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,"Goodbye, Old Friend",Gip Roberts,"['Contest', '#89']",NaN,"['Friendship', 'Drama', 'Fiction']",35,11,"<article class=""font-alt submission-content sp...","April 11, 2021 20:34",116,"[It, was, like, trying, to, swim, the, English...",2041,"{on, I, begging, below, thirties, due, resurfa...",836,0.409603,"[., ., ,, ,, ., ,, ., ., ., ., ,, ,, ., ,, ,, ..."
885,https://blog.reedsy.com/short-story/51rum0/,51rum0,<!DOCTYPE html>\n\n<

extracting and encoding categories

In [380]:
def find_num_cats(row):
    return len(row['categories'].split())

In [381]:
new_df['num_cats'] = new_df.apply(find_num_cats, axis=1)

In [382]:
new_df.head()

,url,story_id,soup,title,author,contest_num,won_contest,categories,num_likes,num_comments,story_html,date_published,num_sent,words,num_words,unique_words,num_unique_words,unique_words_percent,punctuation,num_cats
0,https://blog.reedsy.com/short-story/n1pl7n/,n1pl7n,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,11:59,Krystal Pepper,"['Contest', '#137']",NaN,"['Crime', 'Fiction', 'Thriller']",17,3,"<article class=""font-alt submission-content sp...","March 11, 2022 17:27",125,"[11:59, I, shoot, straight, out, of, bed, to, ...",2645,"{lot, on, I, detective, leave, actually, word,...",774,0.292628,"[., ., ., ., ., ,, ,, ., ,, ,, ., ,, ., ,, ., ...",3
1,https://blog.reedsy.com/short-story/yv1ahb/,yv1ahb,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,Want Ads,Nina Wishnat,"['Contest', '#30']",NaN,"['Contemporary', 'Fiction']",1,0,"<article class=""font-alt submission-content sp...","February 28, 2020 15:27",87,"[Mia, s, room, was, pretty, much, bare, She, l...",1453,"{firmly, better, on, I, made, elevator, With, ...",619,0.426015,"[’, ., ,, ., ., ,, ., ., ., ., ., “, ?, ’, ,, ...",2
2,https://blog.reedsy.com/short-story/kv1cr7/,kv1cr7,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,To be wrong because of sincerity...,Lis Lovén,"['Contest', '#102']",NaN,"['Black', 'Contemporary', 'Fiction']",12,0,"<article class=""font-alt submission-content sp...","July 10, 2021 14:21",150,"[Lin, had, to, figure, out, things, because, o...",2008,"{loads, better, on, creature, word, made, cham...",676,0.336653,"[,, ,, ., ,, ,, ., ., ,, ., ,, ,, ., ,, ,, ., ...",3
3,https://blog.reedsy.com/short-story/v2nqtq/,v2nqtq,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,"The Shiva, 1955",Tammy Kl,"['Contest', '#100']",NaN,"['American', 'Coming', 'of', 'Age', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 01, 2021 18:13",153,"[The, Shiva, 1955, For, the, rest, of, her, li...",2735,"{peeked, on, I, word, fell, polite, made, hold...",994,0.363437,"[,, ,, ., ,, ., –, ,, ,, ,, ’, ., (, !, ), ,, ...",5
4,https://blog.reedsy.com/short-story/nptt18/,nptt18,<!DOCTYPE html>\n\n<html>\n<head>\n<meta chars...,The Monster of Greentree,Best Christopher,"['Contest', '#102']",NaN,"['Coming', 'of', 'Age', 'Adventure', 'Fiction']",6,0,"<article class=""font-alt submission-content sp...","July 17, 2021 01:23",186,"[The, wind, whipped, through, my, hair, as, I,...",2773,"{lot, better, insane, on, I, him—which, word, ...",863,0.311215,"[’, ,, ., ., ,, ,, ., ,, ,, ., ,, ., ,, ,, ’, ...",5


In [383]:
new_df.shape

(887, 20)

In [384]:
import re
cats = set()
for sample, num in new_df[['categories', 'num_cats']].values:
    samp_cats = re.findall('[A-Z][a-z]+', sample)
    cats.update(samp_cats)
print(f'we have {len(cats)} categories')

we have 49 categories


In [385]:
for cat in cats:
    new_df[f'cat_{cat}'] = (new_df['categories']).astype('category').str.contains(cat).astype(int)
new_df.shape

(887, 69)

In [386]:
new_df['cat_Fiction'].sum() == len(new_df)

True

In [387]:
new_df.drop('categories', axis=1, inplace=True)
new_df.shape

(887, 68)

encoding date as datetime

In [388]:
new_df['date_published'] = pd.to_datetime(new_df['date_published'])
new_df['date_published']

In [390]:
new_df.shape

(887, 68)

In [391]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 68 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   url                   887 non-null    object        
 1   story_id              887 non-null    object        
 2   soup                  887 non-null    object        
 3   title                 887 non-null    object        
 4   author                887 non-null    object        
 5   contest_num           887 non-null    object        
 6   won_contest           11 non-null     object        
 7   num_likes             887 non-null    int64         
 8   num_comments          887 non-null    int64         
 9   story_html            887 non-null    object        
 10  date_published        887 non-null    datetime64[ns]
 11  num_sent              887 non-null    object        
 12  words                 887 non-null    object        
 13  num_words           

In [393]:
new_df.describe()

,num_likes,num_comments,num_cats,cat_Western,cat_Adult,cat_East,cat_Color,cat_Christian,cat_School,cat_Kids,...,cat_Urban,cat_People,cat_Creative,cat_Horror,cat_Inspirational,cat_Historical,cat_Science,cat_High,cat_African,cat_Funny
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.00000,887.000000,887.000000,887.000000,...,887.000000,887.00000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000
mean,14.512965,4.717024,2.828636,0.003382,0.032694,0.005637,0.00451,0.006764,0.012401,0.005637,...,0.006764,0.00451,0.003382,0.055242,0.057497,0.018038,0.034949,0.010147,0.002255,0.069899
std,21.576877,21.759873,1.130309,0.058091,0.177936,0.074910,0.06704,0.082013,0.110731,0.074910,...,0.082013,0.06704,0.058091,0.228582,0.232921,0.133165,0.183755,0.100274,0.047458,0.255120
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,14.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,460.000000,420.000000,8.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [396]:
new_df.to_csv('story_data_new_feats.csv', index=False)

In [397]:
other = pd.read_csv('story_data_new_feats.csv')

In [398]:
other.shape

(887, 68)